<a href="https://colab.research.google.com/github/lixinso/sample-intro-to-pytorch/blob/master/multiple_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# https://pytorch.org/tutorials/beginner/former_torchies/parallelism_tutorial.html
# https://pytorch.org/tutorials/beginner/blitz/data_parallel_tutorial.html

In [0]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [0]:
input_size = 5
output_size = 2
batch_size = 30
data_size = 100

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
device

device(type='cpu')

In [0]:
#!nvidia-smi

In [0]:
class RandomDataset(Dataset):

  def __init__(self, size, length):
    self.len = length
    self.data = torch.randn(length, size)

  def __getitem__(self, index):
    return self.data[index]

  def __len__(self):
    return self.len


rand_loader = DataLoader(dataset = RandomDataset(input_size, data_size),
                         batch_size=batch_size, shuffle=True)

In [0]:
class Model(nn.Module):

  def __init__(self, input_size, output_size):
    super(Model, self).__init__()
    self.fc = nn.Linear(input_size, output_size)

  def forward(self, input):
    output = self.fc(input)

    print("\t In Model: input size", input.size(), "output size", output.size())

    return output

In [9]:
model = Model(input_size, output_size)
if torch.cuda.device_count() > 1:
#if True:
  print("Lets use ", torch.cuda.device_count(), " GPUs")
  model = nn.DataParallel(model)

model.to(device)

Model(
  (fc): Linear(in_features=5, out_features=2, bias=True)
)

In [10]:
for data in rand_loader:
  input = data.to(device)
  output = model(input)
  print("OUtside : input size", input.size(), "output_size", output.size)

	 In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
OUtside : input size torch.Size([30, 5]) output_size <built-in method size of Tensor object at 0x7ff92c949090>
	 In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
OUtside : input size torch.Size([30, 5]) output_size <built-in method size of Tensor object at 0x7ff92c949168>
	 In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
OUtside : input size torch.Size([30, 5]) output_size <built-in method size of Tensor object at 0x7ff92c9492d0>
	 In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
OUtside : input size torch.Size([10, 5]) output_size <built-in method size of Tensor object at 0x7ff92c949048>
